In [64]:
import pandas as pd
import numpy as np
import geopandas as gpd
import time, os, fnmatch, shutil

In [65]:
t = time.localtime()
timestamp = time.strftime('%b-%d-%Y_%H%M', t)

In [66]:
mes_curso = 'AI2022-01.xlsx'
df_activity_info = pd.read_excel(r'C:\Users\jorcabrera\OneDrive - International Organization for Migration - IOM\InteragencyGTRM\GTRM 2022\IM 2022\ActivityInfo\DescargasMensuales'+ '\\' + mes_curso)
df_activity_info = df_activity_info[df_activity_info['Value'] != 0]

In [67]:
ruta_dpa_canton_ecuador = "shapefiles/dpa_canton_codificado_v1_4326.shp"
ruta_dpa_provincia_ecuador = "shapefiles/dpa_provincia_codificado4326.shp"
dpa_ecuador_cantones = gpd.read_file(ruta_dpa_canton_ecuador, encoding='utf-8')
dpa_ecuador_provincias = gpd.read_file(ruta_dpa_provincia_ecuador, encoding='utf-8')
dpa_ecuador_cantones = gpd.read_file(ruta_dpa_canton_ecuador, encoding='utf-8')
shapefile_out = "C:/Users/jorcabrera/OneDrive - International Organization for Migration - IOM/Documents/OIM/tmp/"

In [68]:
pivot_canton = df_activity_info.pivot_table(index=['canton.code'],
                                                      values=['Value'],
                                                      aggfunc='count',
                                                      fill_value=0,
                                                      margins_name='Beneficiarios Total',
                                                      margins='True')

In [69]:
flattened_pivot_canton = pd.DataFrame(pivot_canton.to_records())[:-1]
# Renombramos las cabeceras de los campos de la tabla a nombres mas apropiados
flattened_pivot_canton.columns = [hdr.replace("('count', 'Value')", "Value") for hdr in flattened_pivot_canton.columns]


In [70]:
flattened_pivot_canton['canton.code'] = flattened_pivot_canton['canton.code'].str[2:]
flattened_pivot_canton = flattened_pivot_canton.rename(columns={'canton.code': 'DPA_CANTON'})

In [71]:
df_activity_info_geo = pd.merge(dpa_ecuador_cantones, flattened_pivot_canton, how='left', on='DPA_CANTON')
df_activity_info_geo.to_file(driver='ESRI Shapefile', filename= shapefile_out + "2022_Nacional_" + timestamp + "_ACTIVIDADES_CANTON.shp", encoding='utf-8')